# Fine-tune Flux LoRA on the image preferences dataset

Note, we will not use preferences from the dev set for this fine-tuning. We will only use the chosen images for an Supervised fine-tuning phase. Additionally, we recommend using a A100 GPU (4$/hour on Hugging Face) for this fine-tuning because of the memory requirements. The fine-tuning script will take about 4 hours to complete for a single epoch.

## Install dependencies

We first make sure we have the latest version of diffusers installed. This is a development version of diffusers, so we need to install it from source. Additionally, we install the other dependencies that are required for the fine-tuning script.

In [ ]:
!git clone https://github.com/huggingface/diffusers
!pip install -e diffusers/.
!pip install datasets sentencepiece protobuf accelerate peft wandb torchvision prodigyopt

## Logins and config

We will use Weights & Biases to log the training process. Additionally, we log in to Hugging Face to push the finetuned model to the Hub.

In [ ]:
!huggingface-cli login --token "hf_xxx"
!wandb login "xxx"
!accelerate config default

## Fine-tune the model

Lastly, we fine-tune the Flux LoRA on the chosen images from the image preferences dataset. We heavily inspired from the [Dreambooth fine-tuning script](https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/README_flux.md) and modified it to work for our use case.

In [ ]:
!accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_flux.py \
    --pretrained_model_name_or_path "black-forest-labs/FLUX.1-dev" \
    --dataset_name "data-is-better-together/open-image-preferences-v1-binarized" \
    --hub_model_id "davidberenstein1957/open-image-preferences-v1-flux-dev-lora" \
    --push_to_hub \
    --output_dir "open-image-preferences-v1-flux-dev-lora" \
    --image_column "chosen" \
    --caption_column "prompt" \
    --mixed_precision="bf16" \
    --resolution=1024 \
    --train_batch_size=1 \
    --repeats=1 \
    --report_to="wandb"\
    --gradient_accumulation_steps=1 \
    --gradient_checkpointing \
    --learning_rate=1.0 \
    --text_encoder_lr=1.0 \
    --optimizer="prodigy"\
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --rank=8 \
    --checkpointing_steps=2000 \
    --seed="0" 